# Distribution of the data related by the following paper: 

### Investigating the effect of grain structure on compressive response of open-cell metal foam using high-fidelity crystal-plasticity modeling 

__Authors:__  _Dongfang Zhao, Kristoffer E. Matheson, Brian R. Phung, Steve Petruzza, Michael W. Czabaj, Ashley D. Spear_ 

__Published in:__  Materials Science and Engineering, Volume 812, 2021, 140847, ISSN 0921-5093

https://doi.org/10.1016/j.msea.2021.140847

https://www.sciencedirect.com/science/article/pii/S0921509321001167

[Jupyter notebook](https://jupyter.org/)  created by  [Valerio Pascucci](http://cedmav.com/) and  [Giorgio Scorzelli](https://www.sci.utah.edu/people/scrgiorgio.html) 

***
This notebook requires the installation of the following python packages:
* OpenVisus
* pillow
* urllib3
* matplotlib
* mpl_interactions
* ipywidgets
* ipycanvas

The follwing cell attempts to install and configure them using PIP. You may need to do the installation manually if anythign goes wrong or if you use conda.

In [1]:
def InstallPrerequisites():

    # Pre-requirements: install all the python libraries needed to run this exemple.
    # You may need to restart the kernel if the process catches a missing library to be installed. 
    import os,sys

    print("For a manual installation with pip you can use the following comemnds: ")
    print(" ")
    print("{} -m pip  install --upgrade OpenVisus".format(sys.executable))
    print("{} -m OpenVisus configure ".format(sys.executable))

    print("{} -m pip install pillow".format(sys.executable))
    print("{} -m pip install urllib3".format(sys.executable))
    print("{} -m pip install matplotlib".format(sys.executable))
    print("{} -m pip install mpl_interactions".format(sys.executable))
    print("{} -m pip install ipywidgets".format(sys.executable))
    print("{} -m pip install ipycanvas".format(sys.executable))
    print("{} -m jupyter nbextension enable --py widgetsnbextension".format(sys.executable))

    # specific prerequisite (this may need to be completed)
    !"{sys.executable}" -m pip  install --upgrade OpenVisus
    !"{sys.executable}" -m OpenVisus configure 

    # general prerequisites
    !"{sys.executable}" -m pip install pillow
    !"{sys.executable}" -m pip install urllib3
    !"{sys.executable}" -m pip install matplotlib
    !"{sys.executable}" -m pip install mpl_interactions
    !"{sys.executable}" -m pip install ipywidgets
    !"{sys.executable}" -m pip install ipycanvas
    !"{sys.executable}" -m jupyter nbextension enable --py widgetsnbextension

    print("WARNING: you probably need to restart Jupyter")
    
if False:
    InstallPrerequisites()

print("!!!!!!!!!!!!! REMEMBER TO FILE/TRUST NOTEBOOK!!!!!!!!!!!!!!!!!!!!!")

!!!!!!!!!!!!! REMEMBER TO FILE/TRUST NOTEBOOK!!!!!!!!!!!!!!!!!!!!!


In [2]:
%matplotlib inline 

import os,sys,io,random,os,sys,time
import numpy as np
import pandas as pd
import PIL
import matplotlib 
import matplotlib.pyplot as plt

import IPython
import IPython.display
import ipywidgets

from OpenVisus import *
print("current working directory",os.getcwd() )

Starting OpenVisus C:\Python38\lib\site-packages\OpenVisus\__init__.py 3.8.0 (tags/v3.8.0:fa919fd, Oct 14 2019, 19:37:50) [MSC v.1916 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=0, releaselevel='final', serial=0) ...
current working directory C:\projects\nsdf-fabric\notebooks


In [3]:
def Assert(cond):
    if not cond:
        raise Exception("Assert failed")

# //////////////////////////////////////////////////////////////////
class CachedDataset(PyDataset):
    
    # constructor
    def __init__(self, args):
        self.local_filename=os.path.abspath(args["local"]).replace("\\","/")
        self.remote_url=args["url"]
        self.remote_access_type = args["access"]
        self.description=args["description"]
        
        #print("local_filename"    ,self.local_filename)
        #print("remote_url"        ,self.remote_url)
        #print("remote_access_type",self.remote_access_type)
        #print("description",       self.description)
        
        super().__init__(LoadDatasetCpp(self.remote_url))
        
        self.num_blocks = len(self.getFields()) * self.getTotalNumberOfBlocks() * len(self.getTimesteps().asVector())
        self.num_blocks_cached = 0

        self.stop_thread=False
        self.thread=None
        
        self.progress=None
        self.progress_display=None

        #print("Database size",self.getWidth(),self.getHeight(),self.getDepth())
        #print("Fields:",self.getFields()) 
        #print("Loaded cached dataset")
        
    # __del__
    def __del__(self):
        self.stopCaching()   
        
    # createAccess
    def createAccess(self, ):
        
        access_config="""
            <access type='multiplex'>
                    <access type='disk' chmod='rw' url='file://{}' />
                    <access type='{}' url='{}' chmod="r" /> 
            </access>  
        """.format(
            self.local_filename.replace("&","&amp;"),
            self.remote_access_type,
            self.remote_url.replace("&","&amp;")) 
        
        # print("Creating access",access_config)
        access= self.createAccessForBlockQuery(StringTree.fromString(access_config))

        # at this point the cache is enabled with the new local idx file
        Assert(os.path.isfile(self.local_filename))

        return access   

    # startCaching
    def startCaching(self, background=True):
        
        if background:
            self.thread = threading.Thread(target=self.startCaching, args=(False,))
            self.stop_thread=False
            self.thread.start()        
            return 

        #print("start caching","...")
        
        access=self.createAccess()

        access.beginRead()
        
        for field in self.getFields():
            for blockid in range(self.getTotalNumberOfBlocks()): 
                for time in self.getTimesteps().asVector():
                    # print("Copying block","time",time,"field",field,"blockid",blockid,"...")
                    buffer =  self.readBlock(blockid, field=field, time=time, access=access)
                    
                     # to debug missing blocks
                    if  False and buffer is None :
                        read_block = db.createBlockQuery(blockid, ord('r'))
                        msg="# {} {} \n".format(blockid,read_block.getLogicBox().toString())
                        os.write(1, bytes(msg,'utf-8'))                   
                    
                    self.num_blocks_cached += 1
                    self.updateProgress()
                    
                    # stop signal
                    if self.stop_thread:
                        # print("thread stopped")
                        access.endRead()
                        return
                        
        access.endRead()
        self.thread=None
        #print("caching finished done")
        
    # stopCaching
    def stopCaching(self):
        #print("stopping caching...")
        self.stop_thread=True
        if self.thread:
            self.thread.join()
            self.thread=None

    # getWidth
    def getWidth(self):
        p2=self.getLogicBox()[1]
        return p2[0]    
        
    # getHeight
    def getHeight(self):
        p2=self.getLogicBox()[1]
        return p2[1]   
        
    # getDepth
    def getDepth(self):
        p2=self.getLogicBox()[1]
        return p2[2]  
        
    # readSlice
    def readSlice(self,dir=0, slice=0,quality=-3, time=0, access=None):
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W] if dir!=0 else [slice,slice+1]
        y=[0,H] if dir!=1 else [slice,slice+1]
        z=[0,D] if dir!=2 else [slice,slice+1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        width,height=[value for value in ret.shape if value>1]
        return ret.reshape([width,height])
        
    # setProgress
    def setProgress(self,progress, progress_display):
        self.progress=progress
        self.progress_display=progress_display   
        self.progress.min=0
        self.progress.max =self.num_blocks       

    # updateProgress
    def updateProgress(self):     
        
        if self.progress:
            self.progress.value = self.num_blocks_cached
            
        if self.progress_display:
            self.progress_display.value = (
                "Caching progress %.2f%% (%d/%d)" % (
                    100 * self.num_blocks_cached/self.num_blocks, 
                    self.num_blocks_cached,
                    self.num_blocks))                    

            
            
print("Utilities defined")

Utilities defined


Now you can run a background process that slowly copy blocks from remote location

In [4]:
local_cache="./visus-cache/foam/visus.idx"

sources = [
    {
        "url":"https://mghp.osn.xsede.org/vpascuccibucket1/visus-server-foam/visus.idx?compression=zip&layout=hzorder",
        "access":"CloudStorageAccess",
        "local": local_cache,
        "description":'Open Storage Network (OSN) Pod'
    },
    {
        "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=foam&compression=zip&layout=hzorder",
        "access":"network",
        "local": local_cache,
        "description":'University of Utah Campus Server'
    },
    {
        "url" : "https://s3.us-west-1.wasabisys.com/visus-server-foam/visus.idx?compression=zip&layout=hzorder",
        "access":"CloudStorageAccess",
        "local": local_cache,
        "description": 'Wasabi Commercial Cloud Storage'
    },
    # special random==take any of the above
    {
        "url":"random",
        "access":"random",
        "local":"random",
        "description":"random"
    }
]

def PickSource(index):
    N=len(sources)
    if index==N-1:
        index=random.randint(0,N-2)
    return sources[index]

colormaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis','ocean', 'gist_earth', 'terrain', 'gist_stern',
             'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
             'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
             'gist_ncar']

In [5]:
# function to plot the image data with matplotlib
def ShowData(data, cmap=None, plot=None,width = 6):
    ratio=float(data.shape[1])/data.shape[0]
    fig = plt.figure(figsize = (width,width*ratio))
    plot = plt.imshow(data, origin='lower', cmap=cmap)
    return plot

db=CachedDataset(sources[0])
access=db.createAccess()
first_query=db.readSlice(dir=2, slice=512, access=access, time=0, quality=-3)

In [6]:
%matplotlib inline 

from time import sleep
import asyncio
            
# /////////////////////////////////////////////////////////////// 
class Timer:
    
    # constructor
    def __init__(self, delta_sec, callback):
        self.delta_sec = delta_sec
        self._callback = callback
        
    async def _job(self):
        while True:
            await asyncio.sleep(self.delta_sec)
            self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())
        
            
# ///////////////////////////////////////////////////////////////
class Slices:

    # constructor
    def __init__(self, sources,colormaps):
        self.sources=sources
        self.db=None
        self.access=None
        self.currentColormap = colormaps[0]
        self.colormapslist = colormaps
        self.restart_stats=False
        self.refresh_count = 0
        self.refresh_frequency = 10        
        
        # statistics
        self.min=None
        self.max=None        
        self.stats=None
        
        self.threshold=None
        self.updateRange(first_query)
        
        self.createGui()
        
        self.setSource(0)
        self.setDirection(2)

        # incredible: if I start a thread here, pywidgets stop responidng (this is CRAZY)
        if False:
            def Loop():
                while True: time.sleep(1.0)
            thread = threading.Thread(target=Loop)
            thread.run()
            
        timer=Timer(0.3,self.onIdle)
        timer.start()
        
    # onIdle
    def onIdle(self):
        
        logic_box=self.db.getLogicBox()
        dims=[logic_box[1][I]-logic_box[0][I] for I in range(3)]   
        
        if self.stats is None or any([
                self.stats.direction!=self.direction.value,
                self.stats.time!=self.time.value,
                self.stats.threshold!=self.threshold.value]):
        
            class ComputeStats: pass
            self.stats=ComputeStats()
            self.stats.direction=self.direction.value
            self.stats.time=self.time.value
            self.stats.threshold=self.threshold.value
            self.stats.access=self.db.createAccess()
            self.stats.offset=0
            self.stats.x=list(range(dims[self.direction.value]))
            self.stats.y=[0]*len(self.stats.x) 
            self.stats.last_data=None
            self.stats.quality=-15 # TODO
            
        # reached the end?
        if self.stats.offset>=len(self.stats.y):
            return
            
        try:
            data=db.readSlice(
                dir=self.direction.value,
                slice=self.stats.offset,
                quality=self.stats.quality,
                time=self.time.value, 
                access=access)
            self.stats.last_data=data
        except:
            data=self.stats.last_data

        # count non zero pixels
        non_zeros=np.count_nonzero((data>self.threshold.value)*255) 
        self.stats.y[self.stats.offset]=non_zeros
        
        # print("offset",self.stats.offset,"direction",self.direction.value,"threshold",self.threshold.value)

        with self.plot:
            self.plot.clear_output(wait=True)
            fig, axes = plt.subplots()
            if False:
                axes.plot(np.random.normal(size = 50),color="blue")
            else:
                max_x=len(self.stats.x)
                max_y=data.shape[0]*data.shape[1]
                axes.set_xlim([0, max_x])
                axes.set_ylim([0, max_y])
                axes.plot(self.stats.x,self.stats.y,color="blue")
                axes.plot([self.stats.offset, self.stats.offset],[0,max_y],color="gray")
                axes.plot([self.slice.value , self.slice.value ],[0,max_y],color="blue")  
            plt.show(fig) 
        
        self.stats.offset+=1
        

    # createGui
    def createGui(self):    
        layout_width = '60%'
        style = {'description_width': 'initial'}
        
        img=self.convertToImage(first_query)

        self.image=ipywidgets.widgets.Image(
            value=img,
            format='png',
            layout=ipywidgets.widgets.Layout(width=layout_width))
        
        self.source = ipywidgets.widgets.Dropdown(
            options=[(source['description'],I) for I,source in enumerate(self.sources)], 
            value=0,
            description='Data source:',
            style=style,
            layout=ipywidgets.widgets.Layout(width='30%'))
        self.source.observe(lambda __unused__ : self.setSource(self.source.value))

        self.colormapsWidget = ipywidgets.widgets.Dropdown(
            options=self.colormapslist, 
            value=colormaps[0],
            description='Colormap:',
            style=style,
            layout=ipywidgets.widgets.Layout(width='30%'))
        self.colormapsWidget.observe(lambda __unused__ : self.refreshSlice())

        self.time  =ipywidgets.widgets.IntSlider(
            value=0,
            min=0,
            max=3,
            step=1,
            description="time (0-3)",
            layout=ipywidgets.widgets.Layout(width='40%'))
        self.time.observe(lambda __unused__ : self.refreshSlice())

        self.direction = ipywidgets.widgets.Dropdown(
            options=[('X', 0), ('Y', 1), ('Z', 2)], 
            value=0,
            description='Slice orthogonal to axis:',
            style=style,
            layout=ipywidgets.widgets.Layout(width='20%'))
        self.direction.observe(lambda __unused__ : self.setDirection(self.direction.value))

        self.slice=ipywidgets.widgets.IntSlider(
            value=512,
            min=0,
            max=1024,
            step=1,
            description="slice (0-0)",
            layout=ipywidgets.widgets.Layout(width=layout_width))
        self.slice.observe(lambda __unused__ : self.refreshSlice(delayed=True))

        self.resolution =ipywidgets.widgets.IntSlider(
            value=-2,min=-5,max=0,step=1,
            description="Resolution (coarse=-5,full=0)",
            style=style,
            layout=ipywidgets.widgets.Layout(width=layout_width))
        self.resolution.observe(lambda __unused__ : self.refreshSlice())
        
        # stats threshold
        self.threshold =ipywidgets.widgets.IntSlider(
            value=int((self.min+self.max)/2),
            min=self.min,
            max=self.max,
            step=1,
            description="Threshold",
            style=style,
            layout=ipywidgets.widgets.Layout(width=layout_width))   
        self.threshold.observe(lambda __unused__ : self.refreshSlice(delayed=True))
        
        self.info = ipywidgets.widgets.Label("")
        
        self.progress = ipywidgets.widgets.IntProgress(
            min=0, 
            max=0,
            layout=ipywidgets.widgets.Layout(width='70%'))
        self.progress_display = ipywidgets.widgets.Label("Caching progress" +" "*24)
        
        # statistics
        self.plot = ipywidgets.widgets.Output()
   
        # display part
        IPython.display.display(self.image)
        IPython.display.display(ipywidgets.widgets.HBox([self.source, self.colormapsWidget]))
        IPython.display.display(ipywidgets.widgets.HBox([self.direction, self.time  ]))
        IPython.display.display(self.slice)
        IPython.display.display(self.resolution)
        IPython.display.display(self.info)
        IPython.display.display(self.threshold)
        IPython.display.display(ipywidgets.widgets.HBox([self.progress_display, self.progress])) 
        IPython.display.display(self.plot)

    # updateRange
    def updateRange(self,data):
        if self.min is None or self.min > np.amin(data): self.min = np.amin(data)
        if self.max is None or self.max < np.amax(data): self.max = np.amax(data) 
        # update the slider
        if self.threshold:
            self.threshold.min=self.min
            self.threshold.max=self.max            

    # setSource
    def setSource(self,index,cache_data=True): 
        
        # remove previous source db
        if self.db:
            self.db.stopCaching()
            del self.db
            self.db=None
            self.access=None
        
        # create a new source db
        self.db=CachedDataset(PickSource(index))
        self.access=self.db.createAccess()
        if cache_data:
            self.db.startCaching() 
            self.db.setProgress(self.progress,self.progress_display)
        self.refreshSlice() 
        
    # setDirection
    def setDirection(self,value):
        self.direction.value=value
        self.slice.max = self.db.getLogicBox()[1][value]-1        
        self.slice.value = self.slice.max//2
        self.slice.description = "slice (0-{})".format(self.slice.max)   
        self.refreshSlice()

    # refreshSlice
    def refreshSlice(self, delayed=False):
        
        if delayed:
            self.refresh_count = self.refresh_count+1
            if self.refresh_count == self.refresh_frequency:
                self.refresh_count = 0
                self.refreshSlice(delayed=False)
            return
        # euristic to map resolution to slice/quality
        self.currentColormap = self.colormapsWidget.value
        resolution = self.resolution.value
        quality = resolution*3
        size_denominator = int(2**(resolution*-1))
        slice = self.slice.value //size_denominator
        slice = slice *size_denominator
        
        self.info.value = "Time={}, Direction={}, Slice={}  Threshold={} Resolution={} {} , Source={}".format(
            self.time.value,
            ['X', 'Y', 'Z'][self.direction.value],
            slice,
            self.threshold.value, 
            resolution,
            ["(coarsest)","(coarser) ","(coarse)  ","(medium)  ","(fine)    "," (full)   "] [5+resolution],
            self.source.options[self.source.value][0]) 
    
        data = self.db.readSlice(dir=self.direction.value, slice=slice,quality=quality, time=self.time.value, access=self.access)
        if data is None:
            return
        
        self.updateRange(data)
        self.image.value = self.convertToImage(data)
        self.restart_stats=True

    # convertToImage
    def convertToImage(self,data):
        data = (data - self.min)/  (self.max-self.min)
        image = np.uint8(matplotlib.cm.get_cmap(self.currentColormap, 255)(data)*255)
        buffer = io.BytesIO()
        PIL.Image.fromarray(image).save(buffer, format='png')
        return buffer.getvalue()   

slices=Slices(sources,colormaps)
# "Interactive slicing of dataset retrieved from the cloud and cached locally in: "+local_cache

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x10\x00\x00\x02\x00\x08\x06\x00\x00\x00\xd3\xd6U…

IntSlider(value=512, description='slice (0-0)', layout=Layout(width='60%'), max=1024)

IntSlider(value=-2, description='Resolution (coarse=-5,full=0)', layout=Layout(width='60%'), max=0, min=-5, st…

Label(value='')

IntSlider(value=32197, description='Threshold', layout=Layout(width='60%'), max=62523, min=1872, style=SliderS…

Output()